In [1]:
import pandas as pd

In [3]:
pd.read_excel?

In [6]:
allPositions = pd.read_excel('allPositions.xlsx', 'allPositions', skiprows = 1)

In [7]:
allPositions.head()

,证券编码,交易所,证券名称,数量,市值,RM市值,净敞口,多头敞口,空头敞口,CreditSpread,...,产品型1,产品型2,资产类型,行业分类一级,行业分类二级,万德行业分类一级,万德行业分类二级,万德行业分类三级,期权类型,系统源
0,000035.SZ,SZ,中国天楹,17500.0,1.176000e+05,1.176000e+05,1.176000e+05,1.176000e+05,0.0,0.0,...,股票,主板,股票,公用事业,环保工程及服务Ⅱ,工业,商业和专业服务,商业服务与用品,NaN,恒生28
1,000035.SZ,SZ,中国天楹,53000.0,3.561600e+05,3.561600e+05,3.561600e+05,3.561600e+05,0.0,0.0,...,股票,主板,股票,公用事业,环保工程及服务Ⅱ,工业,商业和专业服务,商业服务与用品,NaN,恒生28
2,000046.SZ,SZ,泛海控股,13335849.0,9.388438e+07,9.388438e+07,9.388438e+07,9.388438e+07,0.0,0.0,...,股票,主板,股票,房地产,房地产开发Ⅱ,房地产,房地产Ⅱ,房地产管理和开发,NaN,恒生28
3,000065.SZ,SZ,北方国际,170635.0,2.441787e+06,2.441787e+06,2.441787e+06,2.441787e+06,0.0,0.0,...,股票,主板,股票,建筑装饰,专业工程,工业,资本货物,建筑与工程Ⅲ,NaN,恒生28
4,000333.SZ,SZ,美的集团,1899925.0,1.005250e+08,1.005250e+08,1.005250e+08,1.005250e+08,0.0,0.0,...,股票,主板,股票,家用电器,白色家电,可选消费,耐用消费品与服装,家庭耐用消费品,NaN,恒生28


In [8]:
allPositions.columns

Index(['证券编码', '交易所', '证券名称', '数量', '市值', 'RM市值', '净敞口', '多头敞口', '空头敞口',
       'CreditSpread', 'Idiosyncratic OAS', 'Issuer OAS', 'Beta', 'Gamma',
       'Delta', 'Theta', 'RHO', 'Vega', 'DV01', '衡泰DV01', '衡泰久期', '久期', '修正久期',
       '修正凸性', 'DOLLAR_DELTA', 'DOLLAR_GAMMA', '95%历史VaR', '99%历史VaR',
       '95%历史增量VaR', '99%历史增量VaR', '95%历史成分VaR', '99%历史成分VaR', '95%历史ESVaR',
       '99%历史ESVaR', '95%蒙特卡罗VaR', '99%蒙特卡罗VaR', '95%蒙卡增量VaR', '99%蒙卡增量VaR',
       '95%蒙卡成分VaR', '99%蒙卡成分VaR', '95%蒙卡ESVaR', '99%蒙卡ESVaR', '是否城投债',
       '是否私募债', '发行人评级', '债券评级', '币种', '国家', '资金账号', '多空头', '套保套利', '分级基金',
       '集团', '公司', '部门', '投资策略', '投资组合', '产品大类1', '产品大类2', '产品型1', '产品型2',
       '资产类型', '行业分类一级', '行业分类二级', '万德行业分类一级', '万德行业分类二级', '万德行业分类三级', '期权类型',
       '系统源'],
      dtype='object')

In [11]:
allPositionsAbbr = allPositions[['证券编码',  '证券名称', '数量', 'RM市值', 'Beta', '部门','产品型1', '资产类型', '行业分类一级']]

In [13]:
stocks = allPositionsAbbr[allPositionsAbbr['产品型1'] == '股票']

In [15]:
stocks.dropna()

,证券编码,证券名称,数量,RM市值,Beta,部门,产品型1,资产类型,行业分类一级
0,000035.SZ,中国天楹,17500.0,1.176000e+05,0.0000,金融衍生品业务总部,股票,股票,公用事业
1,000035.SZ,中国天楹,53000.0,3.561600e+05,0.0000,金融衍生品业务总部,股票,股票,公用事业
2,000046.SZ,泛海控股,13335849.0,9.388438e+07,-0.0374,金融衍生品业务总部,股票,股票,房地产
3,000065.SZ,北方国际,170635.0,2.441787e+06,1.0286,金融衍生品业务总部,股票,股票,建筑装饰
4,000333.SZ,美的集团,1899925.0,1.005250e+08,1.5803,证券投资业务总部,股票,股票,家用电器
5,000333.SZ,美的集团,209907.0,1.110618e+07,1.5803,证券投资业务总部,股票,股票,家用电器
6,000333.SZ,美的集团,20000.0,1.058200e+06,1.5803,证券投资业务总部,股票,股票,家用电器
7,000333.SZ,美的集团,3353049.0,1.774098e+08,1.5803,证券投资业务总部,股票,股票,家用电器
8,000403.SZ,ST生化,162547.0,4.954433e+06,0.5545,证券投资业务总部,股票,股票,医药生物
9,000403.SZ,ST生化,478240.0,1.457676e+07,0.5545,证券投资业务总部,股票,股票,医药生物


In [16]:
len(stocks)

509

In [17]:
len(stocks.dropna())

322

In [18]:
stocksA = stocks.dropna()

---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. Use this dataset to answer the questions below.

In [19]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)
df.head()

,№ Summer,01 !,02 !,03 !,Total,№ Winter,01 !.1,02 !.1,03 !.1,Total.1,№ Games,01 !.2,02 !.2,03 !.2,Combined total
Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [20]:
for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [22]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
firstCountry = answer_zero() 
type(firstCountry)
firstCountry

pandas.core.series.Series

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [25]:
import numpy as np

def answer_one():
    maxSG = np.max(df['Gold'])
#     return df.index[df['Gold'] == maxSG]
    return df[df['Gold'] == maxSG].index
a = answer_one()
a
type(a)

Index(['United States'], dtype='object')

pandas.core.indexes.base.Index

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [27]:
def answer_two():
    diffG = np.abs(df['Gold'] - df['Gold.1'])
    maxDiffG = np.max(diffG)
    return df[(df['Gold'] - df['Gold.1']) == maxDiffG].index
answer_two()

Index(['United States'], dtype='object')

### Question 3
Which country has the biggest difference between their summer and winter gold medal counts relative to their total gold medal count? Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [113]:
def answer_three():
    wonGoldCountry = df[df['Gold'] > 0 | df['Gold.1'] > 0]
    relDiffG = np.max(df['Gold'] - df['Gold1'])/(df['Gold'] + df['Gold.1'])
    return df[(df['Gold'] - df['Gold1'])/(df['Gold'] + df['Gold.1']) = relDiffG].index


'Bulgaria'

### Question 4
Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [29]:
def answer_four():
    df['Points'] = df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2']
    return df['Points']
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [30]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [31]:
def answer_five():
    states = census_df['STNAME'].unqiue()
    stateCountyNum = {}
    for state in states:
        countyNum = df[df['STNAME'] == state].count()
        stateCountyNum[state] = countyNum
    return stateCountyNum
answer_five()


AttributeError: 'Series' object has no attribute 'unqiue'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

*This function should return a list of string values.*

In [76]:
def answer_six():


['Wyoming', 'Vermont', 'North Dakota']

### Question 7
Which county has had the largest change in population within the five year period (hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all five columns)?

*This function should return a single string value.*

In [95]:
def answer_seven():


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [106]:
def answer_eight():


,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
